In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# MACHINE LEARNING LIBERARIES

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import re
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

### Loading the dataset

In [3]:
data_tr=pd.read_csv('train.csv')
data_test=pd.read_csv('test.csv')

In [4]:
data_tr.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
data_tr.shape

(7613, 5)

In [6]:
data_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


# DATA PREPROCESSING

In [7]:
del data_tr['keyword']
del data_tr['location']

### Textual Date Cleaning using REGEX

In [8]:
def text_cleaner(text):
    text = text.lower() # convert to lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) # remove punctuation and special characters
    text = re.sub(r'\s+', ' ', text) # remove extra whitespace
    text = re.sub(r'https?://\S+', '', text) # remove URLs
    text = re.sub(r"#", "", text)
    return text

In [9]:
data_tr['text'] = data_tr['text'].apply(text_cleaner)

# ML MODEL BUILDING

In [10]:
x=data_tr['text']
y=data_tr['target']

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=4)

### Converting textual Data into Numeric Form using TF-IDF Vectorization

In [12]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words = 'english')
X_train = vectorizer.fit_transform(x_train)
X_test = vectorizer.transform(x_test)

In [13]:
X_train.shape

(6090, 18739)

### Model Results on best Parameters

### SVM 

In [14]:
modelsvm=svm.SVC()
modelsvm.fit(X_train,y_train)
pred_testsvm=modelsvm.predict(X_test)
accuracy_score(y_test,pred_testsvm)

0.7918581746552856

In [15]:
eval = vectorizer.transform(data_test['text']).toarray()

In [16]:
final_pred = modelsvm.predict(eval)

In [17]:
submission2 = data_test[['id']].reset_index(drop=True)
submission2['target'] = final_pred.astype('int64')

In [18]:
submission2.head(5)

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [19]:
submission2.to_csv('Submission_V6.csv', index=False)